# Multi-Snapshot Network Query Demo
## Load multiple snapshots and query them separately

This notebook demonstrates:
1. Loading multiple snapshots into Neo4j simultaneously
2. Viewing all loaded snapshots
3. Switching between snapshots for queries
4. Comparing network state across different snapshots
5. Deleting specific snapshots when done

In [ ]:
# Setup
import sys
from pathlib import Path
sys.path.append(str(Path().absolute()))

from agents import SnapshotManager, NetworkQueryAgent
import json

## Step 1: Initialize Snapshot Manager

In [ ]:
# Initialize snapshot manager
snapshot_mgr = SnapshotManager()

# Display all available snapshots (from files)
print("Available snapshot files:")
available = snapshot_mgr.display_snapshots()

## Step 2: Load Multiple Snapshots

Load 2-3 different snapshots into Neo4j. They will coexist separately!

In [ ]:
# Load first snapshot
result1 = snapshot_mgr.load_snapshot(snapshot_index=1)
print(f"\n✓ Loaded: {result1['snapshot_id']}")
snapshot_1_id = result1['snapshot_id']

In [ ]:
# Load second snapshot (without clearing!)
result2 = snapshot_mgr.load_snapshot(snapshot_index=2)
print(f"\n✓ Loaded: {result2['snapshot_id']}")
snapshot_2_id = result2['snapshot_id']

In [ ]:
# Optional: Load third snapshot
result3 = snapshot_mgr.load_snapshot(snapshot_index=3)
print(f"\n✓ Loaded: {result3['snapshot_id']}")
snapshot_3_id = result3['snapshot_id']

## Step 3: View All Loaded Snapshots

In [ ]:
# Show all snapshots currently in Neo4j
loaded = snapshot_mgr.display_loaded_snapshots()

print(f"\nTotal snapshots in Neo4j: {len(loaded)}")

## Step 4: Query Specific Snapshots

Create query agents that filter by snapshot

In [ ]:
# Create agent for first snapshot
agent1 = NetworkQueryAgent(snapshot_id=snapshot_1_id)
print(f"Agent 1 queries snapshot: {snapshot_1_id}")

# Create agent for second snapshot  
agent2 = NetworkQueryAgent(snapshot_id=snapshot_2_id)
print(f"Agent 2 queries snapshot: {snapshot_2_id}")

## Step 5: Compare Network State Across Snapshots

Ask the same question to different snapshots

In [ ]:
# Query both snapshots for down interfaces
question = "What interfaces are down?"

print(f"\n{'='*80}")
print(f"Question: {question}")
print(f"{'='*80}")

# Snapshot 1
result1 = agent1.ask(question)
print(f"\n[SNAPSHOT 1] {snapshot_1_id[:25]}...")
print(f"Down interfaces: {result1['count']}")
for i, record in enumerate(result1['results'][:5], 1):  # Show first 5
    print(f"  {i}. {record}")

# Snapshot 2
result2 = agent2.ask(question)
print(f"\n[SNAPSHOT 2] {snapshot_2_id[:25]}...")
print(f"Down interfaces: {result2['count']}")
for i, record in enumerate(result2['results'][:5], 1):  # Show first 5
    print(f"  {i}. {record}")

print(f"\n{'='*80}")
print(f"Difference: {abs(result1['count'] - result2['count'])} interfaces changed status")
print(f"{'='*80}")

In [ ]:
# Compare device lists
question = "Show me all devices"

print(f"\nQuestion: {question}")

result1 = agent1.ask(question)
print(f"\n[SNAPSHOT 1]: {result1['count']} devices")
for record in result1['results']:
    print(f"  - {record}")

result2 = agent2.ask(question)
print(f"\n[SNAPSHOT 2]: {result2['count']} devices")
for record in result2['results']:
    print(f"  - {record}")

## Step 6: Switch Active Snapshot

Change which snapshot is active for the SnapshotManager

In [ ]:
# Set different active snapshot
snapshot_mgr.set_active_snapshot(snapshot_2_id)

# Show current state
snapshot_mgr.display_loaded_snapshots()

## Step 7: Dynamic Snapshot Switching

Use a single agent and change its snapshot filter on the fly

In [ ]:
# Create agent without snapshot filter
dynamic_agent = NetworkQueryAgent()

# Query snapshot 1
dynamic_agent.set_snapshot(snapshot_1_id)
result = dynamic_agent.ask("Count interfaces")
print(f"Snapshot 1 interface count: {result}")

# Switch to snapshot 2
dynamic_agent.set_snapshot(snapshot_2_id)
result = dynamic_agent.ask("Count interfaces")
print(f"Snapshot 2 interface count: {result}")

# Clear filter - query all data
dynamic_agent.clear_snapshot_filter()
result = dynamic_agent.ask("Count interfaces")
print(f"All snapshots interface count: {result}")

## Step 8: Path Comparison Between Snapshots

See if routing changed between snapshots

In [ ]:
# Compare paths between two devices in different snapshots
question = "Show path between EDGE-R1 and ACC-SW1"

print(f"\nQuestion: {question}")

# Snapshot 1 path
result1 = agent1.ask(question)
print(f"\n[SNAPSHOT 1]")
print(f"Paths found: {result1['count']}")
for i, record in enumerate(result1['results'], 1):
    print(f"  Path {i}: {record}")

# Snapshot 2 path
result2 = agent2.ask(question)
print(f"\n[SNAPSHOT 2]")
print(f"Paths found: {result2['count']}")
for i, record in enumerate(result2['results'], 1):
    print(f"  Path {i}: {record}")

if result1['count'] != result2['count']:
    print(f"\n⚠️ WARNING: Number of paths changed!")
else:
    print(f"\n✓ Same number of paths in both snapshots")

## Step 9: Delete a Snapshot

Remove a specific snapshot from Neo4j when you're done with it

In [ ]:
# Delete snapshot 3 (optional)
snapshot_mgr.delete_snapshot(snapshot_3_id)

# View remaining snapshots
snapshot_mgr.display_loaded_snapshots()

## Step 10: Load Same Snapshot Twice (Idempotent)

Loading an already-loaded snapshot won't duplicate data

In [ ]:
# Try loading snapshot 1 again
result = snapshot_mgr.load_snapshot(snapshot_index=1)

if result.get('already_loaded'):
    print("✓ Snapshot was already loaded - no duplication")
else:
    print("Snapshot loaded")

## Summary

**Key Features:**
- ✅ Multiple snapshots loaded simultaneously
- ✅ Each snapshot completely isolated by `snapshot_id`
- ✅ Switch between snapshots without clearing data
- ✅ Compare network state across time
- ✅ Delete specific snapshots
- ✅ Idempotent loading (no duplicates)

**Use Cases:**
1. **Time-series analysis**: Load snapshots from different times and compare
2. **Change tracking**: Identify what changed between snapshots
3. **Testing**: Compare production vs test network states
4. **Rollback planning**: Keep historical snapshots for comparison

## Cleanup

In [ ]:
# Close all connections
agent1.close()
agent2.close()
if 'dynamic_agent' in locals():
    dynamic_agent.close()
snapshot_mgr.close()

print("All connections closed")

## Optional: Clear All Snapshots

If you want to start fresh, clear the entire database

In [ ]:
# Uncomment to clear everything
# snapshot_mgr.clear_neo4j()
# print("Database cleared")